In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
import numpy as np
import pandas as pd
import requests

# Determine the size of each image
from os.path import isfile

%matplotlib inline

In [ ]:
import gdown

In [ ]:
import os
fig = plt.figure(figsize=(8, 8), dpi=100,facecolor='w', edgecolor='k')
train_imgs = os.listdir("../input/humpback-whale-identification/train")
for idx, img in enumerate(np.random.choice(train_imgs, 12)):
    ax = fig.add_subplot(4, 20//5, idx+1, xticks=[], yticks=[])
    im = pil_image.open("../input/humpback-whale-identification/train/" + img)
    plt.imshow(im)

FileNotFoundError: ignored

<Figure size 800x800 with 0 Axes>